In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

ettH1_data = pd.read_csv('ETTh1.csv')
ettH2_data = pd.read_csv('ETTh2.csv')
ettM1_data = pd.read_csv('ETTm1.csv')
ettM2_data = pd.read_csv('ETTm2.csv')

etth1_data = ettH1_data[['date', 'HUFL']].copy()
etth1_data['date'] = pd.to_datetime(etth1_data['date'])

etth2_data = ettH2_data[['date', 'HULL']].copy()
etth2_data['date'] = pd.to_datetime(etth2_data['date'])

ettm1_data = ettM1_data[['date', 'HUFL']].copy()
ettm1_data['date'] = pd.to_datetime(ettm1_data['date'])

ettm2_data = ettM2_data[['date', 'HULL']].copy()
ettm2_data['date'] = pd.to_datetime(ettm2_data['date'])

ettH1_data['date'] = etth1_data['date']
ettH2_data['date'] = etth2_data['date']
ettM1_data['date'] = ettm1_data['date']
ettM2_data['date'] = ettm2_data['date']

etth1_data = ettH1_data[['date', 'HUFL']]
etth1_data['date'] = pd.to_datetime(etth1_data['date'])
etth1_data.set_index('date', inplace=True)
etth1_values = etth1_data.values.astype(float)

etth2_data = ettH2_data[['date', 'HULL']]
etth2_data['date'] = pd.to_datetime(etth2_data['date'])
etth2_data.set_index('date', inplace=True)
etth2_values = etth2_data.values.astype(float)

ettm1_data = ettM1_data[['date', 'HUFL']]
ettm1_data['date'] = pd.to_datetime(ettm1_data['date'])
ettm1_data.set_index('date', inplace=True)
ettm1_values = ettm1_data.values.astype(float)

ettm2_data = ettM2_data[['date', 'HULL']]
ettm2_data['date'] = pd.to_datetime(ettm2_data['date'])
ettm2_data.set_index('date', inplace=True)
ettm2_values = ettm2_data.values.astype(float)

sequence_length = 50

scaler_etth1 = MinMaxScaler()
scaler_etth2 = MinMaxScaler()
scaler_ettm1 = MinMaxScaler()
scaler_ettm2 = MinMaxScaler()

etth1_scaled = scaler_etth1.fit_transform(etth1_values)
etth2_scaled = scaler_etth2.fit_transform(etth2_values)
ettm1_scaled = scaler_ettm1.fit_transform(ettm1_values)
ettm2_scaled = scaler_ettm2.fit_transform(ettm2_values)

def create_lstm_model(sequence_length, num_features):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, num_features)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_multivariate_lstm(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :])
        y.append(data[i + sequence_length, 0])

    X, y = np.array(X), np.array(y)
    model = create_lstm_model(sequence_length, data.shape[1])
    model.fit(X, y, epochs=5, batch_size=32)

    predictions = []
    current_sequence = X[-1]
    for i in range(10 * sequence_length):
        prediction = model.predict(current_sequence.reshape(1, sequence_length, data.shape[1]))[0, 0]
        predictions.append(prediction)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = prediction

    return np.array(predictions)

predictions_etth1 = train_multivariate_lstm(etth1_scaled, sequence_length)
predictions_etth2 = train_multivariate_lstm(etth2_scaled, sequence_length)
predictions_ettm1 = train_multivariate_lstm(ettm1_scaled, sequence_length)
predictions_ettm2 = train_multivariate_lstm(ettm2_scaled, sequence_length)

combined_matrix = np.column_stack((predictions_etth1, predictions_etth2, predictions_ettm1, predictions_ettm2))
print(combined_matrix)



C:\Users\ashis\AppData\Local\Temp\ipykernel_29584\2155850429.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth1_data['date'] = pd.to_datetime(etth1_data['date'])
C:\Users\ashis\AppData\Local\Temp\ipykernel_29584\2155850429.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etth2_data['date'] = pd.to_datetime(etth2_data['date'])
C:\Users\ashis\AppData\Local\Temp\ipykernel_29584\2155850429.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

Epoch 1/5
543/543 [==============================] - 32s 47ms/step - loss: 0.0140
Epoch 2/5
543/543 [==============================] - 22s 41ms/step - loss: 0.0041
Epoch 3/5
543/543 [==============================] - 24s 44ms/step - loss: 0.0030
Epoch 4/5
543/543 [==============================] - 24s 44ms/step - loss: 0.0028
Epoch 5/5
1/1 [==============================] - 0s 26ms/step
Epoch 1/5
543/543 [==============================] - 31s 43ms/step - loss: 0.0031
Epoch 2/5
543/543 [==============================] - 22s 40ms/step - loss: 0.0014
Epoch 3/5
543/543 [==============================] - 21s 39ms/step - loss: 0.0011
Epoch 4/5
543/543 [==============================] - 25s 47ms/step - loss: 0.0010
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
2176/2176 [==============================] - 109s 47ms/step - loss: 0.0023
Epoch 2/5
2176/2176 [==============================] - 102s 47ms/step - loss: 0.0010
Epoch 3/5
2176/2176 [=============================

In [2]:
len_ETTh1 = len(predictions_etth1)
len_ETTh2 = len(predictions_etth2)
len_ETTm1 = len(predictions_ettm1)
len_ETTm2 = len(predictions_ettm2)

print(len_ETTh1, len_ETTh2, len_ETTm1, len_ETTm2)


500 500 500 500


In [3]:
import pandas as pd

date_range = pd.date_range(start='2023-01-01', periods=len_ETTh1, freq='H')

combined_matrix = pd.DataFrame({
    'Date': date_range,
    'Predictions_ETTh1': predictions_etth1,
    'Predictions_ETTh2': predictions_etth2,
    'Predictions_ETTm1': predictions_ettm1,
    'Predictions_ETTm2': predictions_ettm2
})

print(combined_matrix)


                   Date  Predictions_ETTh1  Predictions_ETTh2  \
0   2023-01-01 00:00:00           0.727900           0.545379   
1   2023-01-01 01:00:00           0.758717           0.543408   
2   2023-01-01 02:00:00           0.772171           0.539141   
3   2023-01-01 03:00:00           0.775847           0.534577   
4   2023-01-01 04:00:00           0.776042           0.530192   
..                  ...                ...                ...   
495 2023-01-21 15:00:00           0.948401           0.294034   
496 2023-01-21 16:00:00           0.948442           0.294014   
497 2023-01-21 17:00:00           0.948479           0.293994   
498 2023-01-21 18:00:00           0.948515           0.293974   
499 2023-01-21 19:00:00           0.948553           0.293954   

     Predictions_ETTm1  Predictions_ETTm2  
0             0.740947           0.624651  
1             0.745647           0.628476  
2             0.749147           0.631155  
3             0.750743           0.633040  